[1] 모듈로딩 및 데이터 준비 <hr>

In [2]:
## [1-1] 모듈 로딩
from tkinter import *
import pandas as pd
import numpy as np

import torch
import torchvision
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt

In [3]:
## [1-2] 데이터 준비
import yt_dlp

url = 'https://www.youtube.com/watch?v=gIjrcfxcyoA'

ydl_opts = {"format": "best"}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=False) 
    stream_url = info["url"]

cap = cv2.VideoCapture(stream_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=gIjrcfxcyoA
[youtube] gIjrcfxcyoA: Downloading webpage


[youtube] gIjrcfxcyoA: Downloading android vr player API JSON


[2] YOLO <hr>

In [6]:
## 모델 정의
base_model = YOLO("yolo26n.pt")
up_model = YOLO("yolo26n-seg.pt")

## 자동차, 버스, 트럭, 오토바이 클래스 번호
target_classes = [2, 3, 5, 7]

In [7]:
# while cap.isOpened():
#     ret, frame = cap.read() 
    
#     ## 프레임이 없으면 break
#     if not ret: 
#         break 
    
#     result = model.predict(frame, conf = 0.5, ## 신뢰도 임계값 : 0.5 => 50%이상만 검출 
#                            verbose=False) ## 콘솔로그 출력 끄기 : 영상처리에서 필수
    
#     ## 영상에 bbox 
#     for box in result[0].boxes: 
#         cls_id = int(box.cls[0]) 
#         if cls_id in target_classes: 
#             x1, y1, x2, y2 = map(int, box.xyxy[0]) 
#             conf = float(box.conf[0]) 
            
#             # 네모 박스 
#             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
#             cv2.putText( frame, f"{model.names[cls_id]} {conf:.2f}", 
#                         (x1, y1 - 5), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2 ) 
            
#             cv2.imshow("Vehicle Detection", frame) 
#             if cv2.waitKey(1) & 0xFF == 27: # ESC 
#                 break

In [8]:
# ByteTrack 활성화용 tracker 설정
tracker_cfg = "bytetrack.yaml"
tracker_bst = "bytetrack.yaml"
## tracker_bst = "botsort.yaml"


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_det = frame.copy()
    frame_track = frame.copy()

    # ============================
    # 1️⃣ Detection 전용 (왼쪽)
    # ============================
    results_det = base_model.track(
        frame_det,
        conf=0.5,
        tracker=tracker_cfg,
        persist=True,
        verbose=False
    )

    for box in results_det[0].boxes:
        cls_id = int(box.cls[0])
        if cls_id in target_classes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            track_id = int(box.id[0]) if box.id is not None else -1

            cv2.rectangle(frame_det, (x1,y1), (x2,y2), (0,0,255), 2)
            cv2.putText(frame_det,
                        f"ID:{track_id} {up_model.names[cls_id]} {conf:.2f}",
                        (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0,0,255),
                        2)

    cv2.putText(frame_det, "YOLOv26 + Bytetrack", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    # ============================
    # 2️⃣ Detection + ByteTrack (오른쪽)
    # ============================
    results_track = up_model.track(
        frame_track,
        conf=0.5,
        tracker=tracker_bst,
        persist=True,
        verbose=False
    )

    for box in results_track[0].boxes:
        cls_id = int(box.cls[0])
        if cls_id in target_classes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            track_id = int(box.id[0]) if box.id is not None else -1

            cv2.rectangle(frame_track, (x1,y1), (x2,y2), (0,0,255), 2)
            cv2.putText(frame_track,
                        f"ID:{track_id} {up_model.names[cls_id]} {conf:.2f}",
                        (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0,0,255),
                        2)

    cv2.putText(frame_track, "YOLOv26 + botsort", (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # ============================
    # 3️⃣ 좌우 결합
    # ============================
    combined = np.hstack((frame_det, frame_track))

    cv2.imshow("YOLO Comparison", combined)

    if cv2.waitKey(1) & 0xFF == 27:
        break




In [9]:
cap.release()
cv2.destroyAllWindows()